<a href="https://colab.research.google.com/github/mikodham/intellibots_spring21/blob/main/BaselineCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#REQUIREMENT, #don't forget to turn on gpu
!pip3 install nltk sklearn gensim xgboost keras


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pprint

#PREPROCESSING-Miko
1. Download Dataset
2. Load Dataset
3. Pre-process sentence into features



##1. Load Dataset - REDDIT

In [6]:
# 1st option, upload on ur gdrive
# from google.colab import files
# files.upload()
# 2nd option, just drag the folder from your folder to drive/content/

# 3rd option, git clone
!pwd 
!rm -rf intellibots_spring21 # FOR UPDATING DATASET DO NOT RUN THIS LINE
!git clone https://github.com/mikodham/intellibots_spring21/

/content
Cloning into 'intellibots_spring21'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 103 (delta 43), reused 32 (delta 5), pack-reused 0
Receiving objects: 100% (103/103), 4.31 MiB | 15.18 MiB/s, done.
Resolving deltas: 100% (43/43), done.


###Cleaning The Dataset

In [5]:
DATA_FOLDER = '/content/intellibots_spring21/Dataset/'

def load_data(dataset_name):
    """
    Load data for NN models, with under sampling from Twitter dataset
    :param dataset_name: name of dataset
    :return: X, y
    """
    print('Loading text dataset')
    if dataset_name == 'twitter':
      file_name = DATA_FOLDER + 'sample_data_2.xlsx'
      df = pd.read_excel(file_name)
      df = df.dropna()
      # For balancing #of
      # df_pos = df.loc[df['y'] == 1]
      # df_neg = df.loc[df['y'] == 0]
      # df = pd.concat([df_pos, df_neg.sample(len(df_pos['y']))]) # for balancing
      # df = df.sample(frac=1) # can be used for separating train, test later
      df=df.drop_duplicates() #remove duplicate sentences
      print(df)
      X = df['tweets'].to_numpy()
      y = df['y'].to_numpy()
      return X, y
    elif dataset_name == 'reddit':
      file_name = DATA_FOLDER + 'sample_data_1.xlsx'
      df = pd.read_excel(file_name)
      df = df.dropna()
      df=df.drop_duplicates() #remove duplicate sentences
      print(df)
      x1 = df['title'].to_numpy()
      x2 = df['usertext'].to_numpy()
      y = df['y'].to_numpy()
      return (x1, x2), y
    else:
      raise ValueError("Error: unrecognized dataset")


In [ ]:
X, y = load_data(dataset_name='reddit') # "args_lstm.dataset"
# print(X[:5])

Loading text dataset
                            title  ...  y
0           I want to end my life  ...  1
1  Call suicide hotline for help   ...  0
2                I love this song  ...  0
3  I will commit suicide tomorrow  ...  1

[4 rows x 3 columns]
(array(['I want to end my life', 'Call suicide hotline for help ',
       'I love this song', 'I will commit suicide tomorrow'], dtype=object), array(["I'm a loser. I don't want to live.",
       'Don’t give up. Try to make change. Call the hotline if you need help.',
       'It sounds great.', "I'll never wake up the day after tomorrow."],
      dtype=object))


In [ ]:
# TODO: Find the right data structure to match tensorflow dataset, so that the train and test data
# will be trainable

# Features Extraction

In [ ]:
def f_pos(data, tags_all):
    tag_dict, tag_count, tag_count_body = {}, {}, {}
    for tag in tqdm(tags_all):
        tag_dict[tag] = 0
        tag_count[tag] = []
        tag_count_body[tag] = []
    for title in tqdm(data['title']):
        tagged_text = nltk.pos_tag(nltk.word_tokenize(title))
        for word, tag in tagged_text:
            tag_dict[tag] += 1
        for count, tag in zip(tag_dict.values(), tag_dict.keys()):
            tag_count[tag].append(count)
    for tag in tags_all:
        tag_dict[tag] = 0
    for body in tqdm(data['usertext']):
        for para in body:
            tagged_text = nltk.pos_tag(nltk.word_tokenize(para))
            for word, tag in tagged_text:
                tag_dict[tag] += 1
        for count, tag in zip(tag_dict.values(), tag_dict.keys()):
            tag_count_body[tag].append(count)
    return pd.concat((pd.DataFrame(tag_count, index=None), pd.DataFrame(tag_count_body, index=None)), axis=1)

#RNN

In [ ]:
def build_RNN(args):
    # TODO: Tim, please modify this part use tf.keras instead of keras.model
    # model = tf.keras.Sequential([
    #     tf.keras.layers.Dense(16,activation='relu',input_shape=(NUM_WORDS,)),
    # ])
    inp1 = tf.keras.layers.Input(shape=(args.max_seq_len, ))
    inp2 = tf.keras.kayers.Input(shape=(args.max_seq_len, ))
    x1 = tf.keras.layers.Embedding(args.max_num_words, args.embedding_dim)(inp1)
    x2 = tf.keras.layers.Embedding(args.max_num_words, args.embedding_dim)(inp2)
    x1 = tf.keras.layers.SimpleRNN(args.rnn_units, return_sequences=True)(x1)
    x2 = tf.keras.layers.SimpleRNN(args.rnn_units, return_sequences=True)(x2)
    x = tf.keras.layers.concatenate([x1, x2])
    x = tf.keras.layers.GlobalMaxPool1D()(x)
    x = tf.keras.layers.Dropout(args.dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(args.dense_units, activation="relu")(x)
    x = tf.keras.layers.Dropout(args.dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=[inp1, inp2], outputs=x)


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#LSTM

In [ ]:
import numpy as np
from tqdm import tqdm
from tabulate import tabulate
from sklearn.model_selection import KFold
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, GlobalMaxPool1D, Dropout, BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping

In [ ]:
def build_LSTM(args):
    From XiongJi's Github
    inp = Input(shape=(args.max_seq_len, ))
    # x = Embedding(input_dim=args.max_num_words, output_dim=args.embedding_dim)(inp)
    # x = LSTM(units=args.lstm_units, activation='tanh', dropout=args.dropout_rate, return_sequences=True)(x)
    # x = GlobalMaxPool1D()(x)
    # x = Dropout(args.dropout_rate)(x)
    # x = BatchNormalization()(x)
    # x = Dense(args.dense_units, activation="relu")(x)
    # x = Dropout(args.dropout_rate)(x)
    # x = BatchNormalization()(x)
    # x = Dense(1, activation="sigmoid")(x)

    # TODO: Nabila, please modify this part use tf.keras instead of keras.model
    # model = tf.keras.Sequential([
    #     tf.keras.layers.Dense(16,activation='relu',input_shape=(NUM_WORDS,)),
    # ])

    model = Model(inputs=inp, outputs=x)

    model.compile(
        loss='binary_crossentropy', 
        # OR 'sparse_categorical_crossentropy' for multiclassifcation
        optimizer='adam', 
        # OR 'RMSprop'
        metrics=['accuracy'])
    return model


# RUN For Reddit

In [ ]:
# TODO: Find the right shuffling method and training method

title_train = X1[train_ix]
title_test = X1[test_ix]
usertext_train = X2[train_ix]
usertext_test = X2[test_ix]
y_train = y[train_ix]
y_test = y[test_ix]

tokenizer = text.Tokenizer(num_words=args_rnn.max_num_words)
tokenizer.fit_on_texts(list(title_train)+list(usertext_train))
tokenized_title_train = tokenizer.texts_to_sequences(title_train)
tokenized_title_test = tokenizer.texts_to_sequences(title_test)
tokenized_usertext_train = tokenizer.texts_to_sequences(usertext_train)
tokenized_usertext_test = tokenizer.texts_to_sequences(usertext_test)
X1_train = sequence.pad_sequences(tokenized_title_train, maxlen=args_rnn.max_seq_len, dtype='float64')
X1_test = sequence.pad_sequences(tokenized_title_test, maxlen=args_rnn.max_seq_len, dtype='float64')
X2_train = sequence.pad_sequences(tokenized_usertext_train, maxlen=args_rnn.max_seq_len, dtype='float64')
X2_test = sequence.pad_sequences(tokenized_usertext_test, maxlen=args_rnn.max_seq_len, dtype='float64')

# train model and predict
model = build_RNN(args=args_rnn)
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
model.fit([X1_train, X2_train], y_train, batch_size=args_rnn.batch_size, epochs=args_rnn.epochs,
          validation_split=0.1, callbacks=[early])

y_pred = model.predict([X1_test, X2_test]).reshape(y_test.shape)